In [ ]:
#pip install django
# pip install mysqlclient
# pip install psycopg2-binary
# pip install django-environ

In [ ]:
# django-admin startproject myproject
# python manage.py startapp myapp

#python manage.py makemigrations
#python manage.py migrate

In [ ]:
create project
create app


inside app:
    create urls.py
    create views fbv
    link urls to project urls.py using include 
    create templates and static folders
    add app name in settings.py
    add staticfiles_dirs inside settings


In [ ]:
view urls create for html

view:
def home(request):
    return render(request, 'inside/index.html')

def details(request):
    return render(request, 'inside/detail.html')



path("<post_id>",views.details,name='details')
def details(request,post_id):
    return HttpResponse(post_id)


urls in app:
urlpatterns = [
    path("home/", views.home, name= "home"),
    path("details",views.details,name='details')    
]


link app urls to project urls:
urlpatterns = [
    path('admin/', admin.site.urls),
    path("home/",include("myapp.urls")),
]

In [ ]:
load static file like css,image,js 
# html
{% load static %}
href="{% static 'inside/style.css' %}"

# settings
STATICFILES_DIRS = [
    BASE_DIR / 'myapp' / 'static' / 'inside',
]

In [ ]:
creating layout 
headder fotter base index html

'DIRS': [BASE_DIR / 'templates']

base.html

{% include 'inside/include/header.html' %}

{% block my_content %}
{% endblock %}

{% include 'inside/include/footer.html' %}


index.html

{% extends 'inside/include/base.html' %}

{% block my_content %}
    html codes
{% endblock %}

# Django Filters

**Key takeaway:** Django’s filtering system provides a powerful, expressive API for querying and manipulating data both in views (using the ORM) and in templates. Mastery of `QuerySet` filters, lookups, `Q` objects, and template filters will greatly streamline data retrieval and presentation.

## 1. ORM Filtering with QuerySets

### 1.1. Basic `.filter()` and `.exclude()`
- **`.filter(**kwargs)**:** Returns a new `QuerySet` containing objects that match the given lookup parameters.
- **`.exclude(**kwargs)**:** Returns a `QuerySet` excluding objects that match the lookup parameters.

Example:
```python
from myapp.models import Book

# All books published in 2024
books_2024 = Book.objects.filter(publication_year=2024)

# All books not in 2024
books_not_2024 = Book.objects.exclude(publication_year=2024)
```

### 1.2. Field Lookups
Django supports a variety of lookups by appending a lookup type after a double underscore (`__`):

| Lookup         | Description                                           |
|----------------|-------------------------------------------------------|
| `exact`        | Exact match                                           |
| `iexact`       | Case-insensitive exact match                         |
| `contains`     | Contains substring                                    |
| `icontains`    | Case-insensitive contains                             |
| `gt` / `gte`   | Greater than / greater than or equal to               |
| `lt` / `lte`   | Less than / less than or equal to                     |
| `startswith`   | Starts with                                           |
| `istartswith`  | Case-insensitive starts with                          |
| `endswith`     | Ends with                                             |
| `iendswith`    | Case-insensitive ends with                            |
| `range`        | Within a given range (inclusive)                      |
| `in`           | In a list of values                                   |
| `isnull`       | Is or is not `NULL`                                   |
| `regex`        | Matches a regular expression                          |

Example:
```python
# Books whose title contains "Django" (case-insensitive)
django_books = Book.objects.filter(title__icontains="django")

# Users aged between 18 and 30
young_adults = User.objects.filter(age__range=(18, 30))

# Posts published in the last 7 days
from datetime import timedelta, date
recent_posts = Post.objects.filter(published_date__gte=date.today() - timedelta(days=7))
```

### 1.3. Chaining Filters
Filters can be chained to refine querysets:
```python
# Books published in 2023 with titles containing "API"
api_books_2023 = Book.objects.filter(publication_year=2023).filter(title__icontains="API")
```

## 2. Complex Queries with Q Objects

When you need to combine filters with OR logic or negate nested conditions, use `django.db.models.Q`.

```python
from django.db.models import Q

# Books published in 2022 OR whose author is "Alice"
books_2022_or_alice = Book.objects.filter(
    Q(publication_year=2022) | Q(author__name="Alice")
)

# Books not published in 2021 AND title contains "Guide"
books_not_2021_guide = Book.objects.filter(
    ~Q(publication_year=2021), title__icontains="guide"
)
```

## 3. Aggregation and Annotation Filters

Combine filtering with aggregation to compute summaries:

```python
from django.db.models import Count

# Authors with more than 5 books
prolific_authors = Author.objects.annotate(book_count=Count('book')).filter(book_count__gt=5)
```

## 4. Template Filters

Django templates include built-in filters to transform data for display. Use the pipe (`|`) syntax.

### 4.1. Common Template Filters

| Filter      | Description                            |
|-------------|----------------------------------------|
| `lower`     | Converts string to lowercase           |
| `upper`     | Converts string to uppercase           |
| `title`     | Title-cases string                     |
| `default`   | Provides default value if variable is empty |
| `date`      | Formats a date                         |
| `pluralize` | Appends ‘s’ if value is not 1          |
| `truncatechars` | Truncates string to specified length |

Example:
```django
{{ user.username|upper }}
{{ article.content|truncatechars:100 }}
{{ value|default:"N/A" }}
{{ post.published_date|date:"F j, Y" }}
```

### 4.2. Custom Template Filters

Define reusable filters in your app:

```python
# myapp/templatetags/custom_filters.py
from django import template

register = template.Library()

@register.filter
def seconds_to_hms(value):
    hours = value // 3600
    minutes = (value % 3600) // 60
    seconds = value % 60
    return f"{hours}h {minutes}m {seconds}s"
```

Usage in template:
```django
{% load custom_filters %}
Elapsed time: {{ video.duration|seconds_to_hms }}
```

## 5. Best Practices

- **Select only needed fields:** Use `.only()` or `.values()` to limit data fetched.
- **Index frequently filtered fields:** Improves query performance for large tables.
- **Avoid N+1 queries:** Use `select_related()` (for foreign keys) and `prefetch_related()` (for many-to-many or reverse relations).
- **Combine logic when possible:** Chaining filters is preferable to multiple `.filter()` calls that can be joined into one.

***

By leveraging Django’s ORM filters, `Q` objects, aggregation, and template filters, you can write concise, readable, and performant data queries and presentation logic throughout your application.

# Django Template Filters - Complete Detailed Guide

Django template filters are powerful tools used to transform and format data in HTML templates. They are applied using the pipe (`|`) character and can be chained to perform multiple transformations.

## Basic Syntax
```django
{{ variable|filter_name }}
{{ variable|filter_name:"argument" }}
{{ variable|filter1|filter2:"argument" }}
```

## Complete List of Django Built-in Template Filters

### **String Manipulation Filters**

#### **add**
Adds the argument to the value (works with numbers, strings, and lists).
```django
{{ value|add:"2" }}
<!-- If value is 4, output is 6 -->
{{ first_list|add:second_list }}
<!-- Concatenates two lists -->
```

#### **addslashes**
Adds slashes before quotes for escaping strings.
```django
{{ value|addslashes }}
<!-- If value is "I'm Django", output is "I\'m Django" -->
```

#### **capfirst**
Capitalizes the first character of the value.
```django
{{ value|capfirst }}
<!-- If value is "django", output is "Django" -->
```

#### **center**
Centers the value in a field of given width.
```django
{{ value|center:"15" }}
<!-- If value is "Django", output is "    Django    " -->
```

#### **cut**
Removes all occurrences of the argument from the string.
```django
{{ value|cut:" " }}
<!-- If value is "String with spaces", output is "Stringwithspaces" -->
```

#### **lower**
Converts string to lowercase.
```django
{{ value|lower }}
<!-- If value is "DJANGO", output is "django" -->
```

#### **upper**
Converts string to uppercase.
```django
{{ value|upper }}
<!-- If value is "django", output is "DJANGO" -->
```

#### **title**
Converts string to title case (first letter of each word capitalized).
```django
{{ value|title }}
<!-- If value is "hello world", output is "Hello World" -->
```

#### **slugify**
Converts to lowercase, removes non-word characters, and converts spaces to hyphens.
```django
{{ value|slugify }}
<!-- If value is "Hello World!", output is "hello-world" -->
```

#### **stringformat**
Formats the variable according to the format string.
```django
{{ value|stringformat:"02d" }}
<!-- If value is 5, output is "05" -->
```

### **Text Processing Filters**

#### **truncatechars**
Truncates a string after a certain number of characters.
```django
{{ value|truncatechars:10 }}
<!-- If value is "Hello World", output is "Hello W..." -->
```

#### **truncatewords**
Truncates a string after a certain number of words.
```django
{{ value|truncatewords:2 }}
<!-- If value is "Hello world everyone", output is "Hello world ..." -->
```

#### **truncatechars_html**
Similar to truncatechars but is aware of HTML tags.
```django
{{ value|truncatechars_html:10 }}
<!-- Truncates but doesn't break HTML tags -->
```

#### **truncatewords_html**
Similar to truncatewords but is aware of HTML tags.
```django
{{ value|truncatewords_html:2 }}
<!-- Truncates words but preserves HTML structure -->
```

#### **wordcount**
Returns the number of words in a string.
```django
{{ value|wordcount }}
<!-- If value is "Hello world", output is 2 -->
```

#### **wordwrap**
Wraps words at specified line length.
```django
{{ value|wordwrap:5 }}
<!-- Wraps lines at 5 characters -->
```

### **HTML and Safety Filters**

#### **escape**
Escapes HTML characters in the variable.
```django
{{ value|escape }}
<!-- If value is "<script>", output is "&lt;script&gt;" -->
```

#### **force_escape**
Forces HTML escaping on a variable.
```django
{{ value|force_escape }}
<!-- Always escapes HTML, even if marked as safe -->
```

#### **safe**
Marks a string as safe for HTML output (disables auto-escaping).
```django
{{ value|safe }}
<!-- Outputs HTML without escaping -->
```

#### **safeseq**
Applies the safe filter to each element of a sequence.
```django
{{ html_list|safeseq }}
<!-- Marks each item in the list as safe -->
```

#### **striptags**
Removes all HTML tags from a string.
```django
{{ value|striptags }}
<!-- If value is "<p>Hello</p>", output is "Hello" -->
```

#### **linebreaks**
Converts line breaks to `<p>` and `<br>` HTML tags.
```django
{{ value|linebreaks }}
<!-- Converts newlines to HTML paragraph tags -->
```

#### **linebreaksbr**
Converts line breaks to `<br>` HTML tags only.
```django
{{ value|linebreaksbr }}
<!-- Converts newlines to <br> tags -->
```

#### **urlize**
Converts URLs in plain text into clickable links.
```django
{{ value|urlize }}
<!-- Converts "Visit https://example.com" to HTML link -->
```

#### **urlizetrunc**
Like urlize but truncates URLs longer than the given character limit.
```django
{{ value|urlizetrunc:15 }}
<!-- Creates links but truncates long URLs -->
```

### **List and Data Structure Filters**

#### **first**
Returns the first item in a list.
```django
{{ list|first }}
<!-- If list is ['a', 'b', 'c'], output is 'a' -->
```

#### **last**
Returns the last item in a list.
```django
{{ list|last }}
<!-- If list is ['a', 'b', 'c'], output is 'c' -->
```

#### **join**
Joins a list with a string delimiter.
```django
{{ list|join:", " }}
<!-- If list is ['a', 'b', 'c'], output is "a, b, c" -->
```

#### **length**
Returns the length of a string or list.
```django
{{ list|length }}
<!-- If list has 3 items, output is 3 -->
```

#### **length_is**
Returns True if the length equals the argument.
```django
{{ list|length_is:"3" }}
<!-- Returns True if list has exactly 3 items -->
```

#### **make_list**
Converts a value into a list of characters.
```django
{{ value|make_list }}
<!-- If value is "abc", output is ['a', 'b', 'c'] -->
```

#### **random**
Returns a random item from a list.
```django
{{ list|random }}
<!-- Returns a random element from the list -->
```

#### **slice**
Returns a slice of a list.
```django
{{ list|slice:":2" }}
<!-- Returns first 2 items of the list -->
```

#### **unordered_list**
Recursively takes a nested list and returns an HTML unordered list.
```django
{{ nested_list|unordered_list }}
<!-- Converts nested list to HTML <ul><li> structure -->
```

### **Date and Time Filters**

#### **date**
Formats a date according to the given format string.
```django
{{ value|date:"F j, Y" }}
<!-- If value is datetime, output is "January 1, 2024" -->
{{ value|date:"Y-m-d" }}
<!-- Output is "2024-01-01" -->
```

#### **time**
Formats a time according to the given format string.
```django
{{ value|time:"H:i" }}
<!-- If value is datetime, output is "14:30" -->
```

#### **timesince**
Returns the time since the given date as a human-readable string.
```django
{{ value|timesince }}
<!-- If value was 2 hours ago, output is "2 hours" -->
{{ value|timesince:other_date }}
<!-- Time difference between two dates -->
```

#### **timeuntil**
Returns the time until the given date as a human-readable string.
```django
{{ value|timeuntil }}
<!-- If value is in 3 days, output is "3 days" -->
```

### **Numeric Filters**

#### **floatformat**
Displays a float with a specified number of decimal places.
```django
{{ value|floatformat }}
<!-- Default: 1 decimal place -->
{{ value|floatformat:2 }}
<!-- 2 decimal places -->
{{ value|floatformat:"-2" }}
<!-- 2 decimal places, no trailing zeros -->
```

#### **get_digit**
Returns the requested digit of a number (1-indexed from the right).
```django
{{ value|get_digit:2 }}
<!-- If value is 123456, output is 5 (2nd digit from right) -->
```

### **Logic and Conditional Filters**

#### **default**
If value evaluates to False, uses the given default.
```django
{{ value|default:"N/A" }}
<!-- If value is empty or False, output is "N/A" -->
```

#### **default_if_none**
If value is None, uses the given default.
```django
{{ value|default_if_none:"Unknown" }}
<!-- If value is None, output is "Unknown" -->
```

#### **divisibleby**
Returns True if the value is divisible by the argument.
```django
{{ value|divisibleby:"3" }}
<!-- Returns True if value is divisible by 3 -->
```

#### **yesno**
Maps values for True, False, and (optionally) None to strings.
```django
{{ value|yesno:"Yes,No,Maybe" }}
<!-- True→"Yes", False→"No", None→"Maybe" -->
{{ value|yesno:"Yes,No" }}
<!-- True→"Yes", False→"No" -->
```

### **Special Purpose Filters**

#### **filesizeformat**
Formats a number of bytes as a human-readable file size.
```django
{{ value|filesizeformat }}
<!-- If value is 1024, output is "1.0 KB" -->
```

#### **phone2numeric**
Converts phone numbers with letters into numeric equivalents.
```django
{{ value|phone2numeric }}
<!-- Converts "1-800-COLLECT" to "1-800-2655328" -->
```

#### **pluralize**
Returns a plural suffix if the value is not 1.
```django
{{ items|length }} item{{ items|length|pluralize }}
<!-- If 1 item: "1 item", if 2+ items: "2 items" -->
{{ items|length }} cand{{ items|length|pluralize:"y,ies" }}
<!-- If 1: "1 candy", if 2+: "2 candies" -->
```

#### **pprint**
Pretty-prints a variable for debugging.
```django
{{ complex_object|pprint }}
<!-- Formats complex data structures for easy reading -->
```

### **Text Alignment Filters**

#### **ljust**
Left-aligns the value in a field of given width.
```django
{{ value|ljust:"10" }}
<!-- Left-aligns in 10-character field -->
```

#### **rjust**
Right-aligns the value in a field of given width.
```django
{{ value|rjust:"10" }}
<!-- Right-aligns in 10-character field -->
```

### **Advanced Filters**

#### **dictsort**
Sorts a list of dictionaries by the given key.
```django
{{ dict_list|dictsort:"name" }}
<!-- Sorts list of dictionaries by 'name' key -->
```

#### **dictsortreversed**
Sorts a list of dictionaries in reverse order by the given key.
```django
{{ dict_list|dictsortreversed:"age" }}
<!-- Sorts in reverse by 'age' key -->
```

#### **json_script**
Safely outputs a Python object as JSON, wrapped in script tags.
```django
{{ data|json_script:"my-data" }}
<!-- Outputs: <script id="my-data" type="application/json">...</script> -->
```

#### **linenumbers**
Displays text with line numbers.
```django
{{ value|linenumbers }}
<!-- Adds line numbers to each line of text -->
```

#### **urlencode**
URL-encodes a string.
```django
{{ value|urlencode }}
<!-- Converts special characters for URL use -->
```

## Filter Chaining

Filters can be chained together to apply multiple transformations:

```django
{{ value|lower|truncatechars:10|title }}
<!-- Converts to lowercase, truncates to 10 chars, then title case -->

{{ text|escape|linebreaks }}
<!-- Escapes HTML then converts linebreaks to <p> tags -->

{{ date_value|date:"Y-m-d"|default:"No date" }}
<!-- Formats date, with fallback if None -->
```

## Best Practices

1. **Use appropriate filters**: Choose the right filter for your data type and use case.[1][2][3][4]

2. **Chain filters logically**: Order matters when chaining filters - apply transformations in the correct sequence.[3][4]

3. **Handle None values**: Use `default` or `default_if_none` filters to handle empty or None values gracefully.[2][1]

4. **Be mindful of escaping**: Use `safe` filter only when you trust the content, otherwise rely on Django's auto-escaping.[2]

5. **Custom filters for complex logic**: Create custom template filters for complex transformations that aren't covered by built-ins.[5][6]

This comprehensive list covers all the major built-in Django template filters available for use in HTML templates, providing you with powerful tools to format and transform your data presentation.

[1](https://www.w3schools.com/django/ref_tags_filter.php)
[2](https://docs.djangoproject.com/en/5.2/ref/templates/builtins/)
[3](https://www.geeksforgeeks.org/python/django-template-filters/)
[4](https://earthly.dev/blog/django-template-filters/)
[5](https://aisaastemplate.com/blog/django-template-tags-filters-guide/)
[6](https://docs.djangoproject.com/en/5.2/howto/custom-template-tags/)
[7](https://docs.djangoproject.com/en/5.2/topics/templates/)
[8](https://www.djangotemplatetagsandfilters.com)
[9](https://www.w3schools.com/django/ref_filters_add.php)
[10](https://www.w3schools.com/django/ref_filters_title.php)
[11](https://www.w3schools.com/django/ref_filters_make_list.php)
[12](https://docs.djangoproject.com/en/5.2/ref/templates/language/)
[13](https://stackoverflow.com/questions/72501465/filter-categories-from-list-of-products-in-django-templates)

You can definitely store multiple sets of data in a single `.env` file, including multiple database connection credentials, and use django-environ to manage them efficiently.

## Using Multiple Database Configurations in One .env File

### How to Store Multiple Databases

You can define environment variables for each database connection separately in the `.env` file, for example:

```
# Primary database (default)
DB1_ENGINE=django.db.backends.postgresql
DB1_NAME=primary_db
DB1_USER=primary_user
DB1_PASSWORD=primary_password
DB1_HOST=localhost
DB1_PORT=5432

# Secondary database
DB2_ENGINE=django.db.backends.mysql
DB2_NAME=secondary_db
DB2_USER=secondary_user
DB2_PASSWORD=secondary_password
DB2_HOST=localhost
DB2_PORT=3306
```

### How to Use in Django `settings.py`

You can load both sets and configure Django’s `DATABASES` dictionary with multiple databases:

```python
DATABASES = {
    'default': {
        'ENGINE':   env('DB1_ENGINE'),
        'NAME':     env('DB1_NAME'),
        'USER':     env('DB1_USER'),
        'PASSWORD': env('DB1_PASSWORD'),
        'HOST':     env('DB1_HOST'),
        'PORT':     env('DB1_PORT'),
    },
    'secondary': {
        'ENGINE':   env('DB2_ENGINE'),
        'NAME':     env('DB2_NAME'),
        'USER':     env('DB2_USER'),
        'PASSWORD': env('DB2_PASSWORD'),
        'HOST':     env('DB2_HOST'),
        'PORT':     env('DB2_PORT'),
    },
}
```

## Purpose of django-environ

- **Secure configuration management:** Keeps sensitive data like passwords out of source code.
- **Centralized environment handling:** Simplifies managing development, staging, and production settings.
- **Type casting and defaults:** Allows automatic type conversion (e.g., bool, int) and default values.
- **Ease of use:** Reads `.env` files and environment variables seamlessly in one place.

## Summary

- You **do not need multiple `.env` files** for multiple databases; a single `.env` can hold all variables.
- Use **prefixes** or naming conventions (e.g., `DB1_`, `DB2_`) for clarity.
- Load variables via `env()` in `settings.py` to configure multiple databases.

This approach keeps your configurations flexible, secure, and maintainable.

In [20]:
from pathlib import Path
import environ
import os

BASE_DIR = Path(r'E:\DATA ANALYTICS\GitHub Repo\My Repo\Others\Python\Webframework\myproject')

env_path = BASE_DIR / '.env' 

env = environ.Env()
environ.Env.read_env(env_path)

print(env('SECRET_KEY'))  # No backslash in variable name here!


django-insecure-&1g)+%hhn3l&946)*g6x3lzt#thm8@&yp4nbfbg+f0e-+ty2l2


In [15]:
ls

 Volume in drive E is Rajesh
 Volume Serial Number is A808-6776

 Directory of e:\DATA ANALYTICS\GitHub Repo\My Repo\Others\Python\Webframework

27-08-2025  01:56    <DIR>          .
25-08-2025  17:51    <DIR>          ..
27-08-2025  01:56    <DIR>          .venv
27-08-2025  01:24    <DIR>          bootstrap-blog-templates-main
27-08-2025  18:30    <DIR>          myproject
27-08-2025  19:08            40,233 notes.ipynb
               1 File(s)         40,233 bytes
               5 Dir(s)  127,998,287,872 bytes free


In [ ]:
print("hello")

hello
